In [ ]:
##############################################################################
# Created by: Wesley DA SILVA COELHO
##############################################################################

##############################################################################
#                   INCLUDE PACKAGES
##############################################################################
using NBInclude #Package to include personal files",
using JuMP #Mathematical programming package : allows us to define the MILP",
using LightGraphs #Package to construct simple graphs",
using MetaGraphs #Package to construct complex graphs",
using GraphPlot

# path to some folders
input_folder = "../instances/heuristic/tiny/"
include_folder = "../include/"
result_folder = "../instances/heuristic/tiny/results/" 
@nbinclude(joinpath(include_folder,"NSDP_instance_generator.ipynb"))# code creator for 5GNSD instances
@nbinclude(joinpath(include_folder,"NSDP_structures.ipynb")) # our data structures 
@nbinclude(joinpath(include_folder,"NSDP_instance_reader.ipynb"))# file responsible for parseing new NSDP instances
@nbinclude(joinpath(include_folder,"NSDP_heuristic.ipynb")) # impementation of our model 



my_VIs = ["none"]
my_cuts = ["none"]
my_solvers = ["CPLEX"]
my_OFs = ["minNFS"]
boosts = [true]
my_parameters = Parameters()

my_instance_sizes = get_my_instance_sizes()
latency = ["low","high"]
isolation = ["strong","weak"]
capacities = ["tight","moderate"]
instances_type = 0
exp_re = 30


for instance_size in 1:1
    number_of_instances = 0
    for lat in latency, iso in isolation, c in capacities

        size = my_instance_sizes[instance_size]
        instances_type = instances_type + 1
        n = size["nodes"]
        d = size["densitiy"]
       if  iso == "strong"
            nfsh  = 0.1
            nosh = 0.1
        else
            nfsh  = 0.75
            nosh = 0.75
        end

        s  = size["slices"]
        k = size["commodities"]
        dp = size["DP_NFSs"]
        cp = size["CP_NFSs"]

        tries = 0
        i = 0

        while i <30
            i = i+1

            #creating instances
            number_of_instances = number_of_instances+1
            instance = get_Instance(input_folder,number_of_instances)  
            for of in my_OFs, b in boosts, sol in my_solvers, vi in my_VIs,cut in my_cuts
                my_parameters = Parameters()
                my_parameters.class = "$(size["class"])_$(c)_$(lat)_$(iso)"            
                my_parameters.instance_name = "instance_$(number_of_instances)_of_$(of)_boost_$(b)_solver_$(sol)_vi_$(vi)_cut_$(cut)_$(my_parameters.class)"              
                my_parameters.valid_inequalities = vi
                my_parameters.boost = b
                #my_parameters.instance_class = ""            
                my_parameters.solver = sol
                my_parameters.instance_type = "instance_$(number_of_instances)_type_$(instances_type)_of_$(of)"
                my_parameters.instance_number = number_of_instances
                my_parameters.warming_up = false
                my_parameters.number_of_phy_nodes = n
                my_parameters.graph_density = d
                my_parameters.link_capacity_ratio = c
                my_parameters.node_capacity_ratio =c 
                my_parameters.number_of_slice_requests =s
                my_parameters.number_of_commodities_per_NS = k
                my_parameters.number_of_CP_NFSs = cp 
                my_parameters.number_of_DP_NFSs = dp
                my_parameters.prob_sharing_NFS = nfsh
                my_parameters.prob_sharing_node = nosh
                my_parameters.objf = of                
                my_parameters.cut_class = cut
                my_parameters.cut_time = 0.0
                my_parameters.number_of_cuts = 0
                my_parameters.number_of_lazy_cuts= 0
                my_parameters.lazy_time = 0.0            


        #creating model

                #recreating new model
                my_parameters.relaxation=false
                model = create_NSDP_complete_model(instance,my_parameters)                
                # calling solver 
                solve_NSDP_model(model,my_parameters,instance,"")
                export_stat_instance(my_parameters,exp_re,"complete")
                get_solution_mod(model,instance,result_folder,lat,number_of_instances,my_parameters)
                model = nothing 
                my_instance = solve_NSDP_heuristic(input_folder,number_of_instances, my_parameters.MILP_solution)
                export_stat_instance_heu(my_parameters,exp_re,my_instance,my_parameters.MILP_solution)
                get_solution_heu(my_instance,result_folder,lat,number_of_instances,my_parameters)
            end
        end
    end
end    
